In [1]:
import numpy as np

In [2]:
s = np.loadtxt('new_edges_20000.txt') # edge file

In [8]:
x = s[:,0]
y = s[:,1]
max = np.max(x).astype(np.uint32)

In [9]:
c = 0
for i in range(max+1):
  if i not in x and i not in y:
    c = c+1

In [10]:
nodes = np.zeros((max+1-c,1))
f =0
for i in range(max+1):
  if i in x or i in y:
    nodes[f] = i
    f = f+1

In [11]:
nodes = nodes.astype(np.uint32)

In [12]:
s = s.astype(np.uint32)

In [13]:
adj_list = {}
for i in range(nodes.shape[0]):
  adj_list[nodes[i][0]] = []
for i in range(1,s.shape[0]):
  adj_list[s[i][0]].append(s[i][1])
  #adj_list[s[i][1]].append(s[i][0])

In [15]:
r = np.loadtxt('barbell_20000_128.emb')       #embedding file

In [16]:
emb = {}
for i in range(nodes.shape[0]):
  emb[r[i,0]] = r[i,1:] 

In [17]:
import random
pos_edges = random.sample(range(s.shape[0]), 150000)

In [19]:
c=0
neg_edges = np.zeros((150000,2))
while(c<150000):
  l = (random.sample(list(nodes[:,0]),1)[0], random.sample(list(nodes[:,0]),1)[0])
  if l[1] not in adj_list[l[0]]:
    neg_edges[c][0] = l[0]
    neg_edges[c][1] = l[1]
    c=c+1
    if c%10000==0:
        print(c)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000


In [21]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu'))
#model.add(Dense(256, activation='relu'))
#model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])preds = (pred>=0.5)*1

In [ ]:
X_train = np.zeros((300000,256))
Y_train = np.zeros((300000,))
for i in range(150000):
  X_train[2*i,:] = np.concatenate((emb[s[pos_edges[i]][0]],emb[s[pos_edges[i]][1]]))
  X_train[2*i+1,:] = np.concatenate((emb[neg_edges[i][0]],emb[neg_edges[i][1]]))
  Y_train[2*i] = 1
  Y_train[2*i+1] = 0

In [ ]:
model.fit(X_train_train,Y_train_train,batch_size=128,epochs=7,validation_split = 0.2)

In [ ]:
pred = model.predict(X_train_test)

In [ ]:
preds = (pred>=0.5)*1

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_train_test, preds)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_train_test, preds))